# Satellite Link Analysis tool
## Written by Cyril Cadoux and Léa Bommottet for the satellite communication class (2018)
### The main cells are located at the end of this notebook

In [1]:
import math
import numpy as np
from scipy.linalg import toeplitz

## Let's define a few constants

In [48]:
EARTH_RADIUS = 6378*1000 # m
GEO_RADIUS = 42174*1000 # m, distance between geostationary satellite and center of the EARTH
GEO_ALTITUDE = GEO_RADIUS - EARTH_RADIUS

LEO_ALTITUDE = 1000*1000 # m
LEO_RADIUS = LEO_ALTITUDE + EARTH_RADIUS

## Let's define utility funtions

In [86]:
'''Let's define utility functions'''

def to_deg(rad):
    return rad*180/math.pi

def to_rad(deg):
    return deg*math.pi/180

def to_dB(decimal):
    return 10*math.log10(decimal)

def to_decimal(dB):
    return 10**(dB/10)

def cos_deg(deg):
    return math.cos(to_rad(deg))

def sin_deg(deg):
    return math.sin(to_rad(deg))

## Now let's define getter functions for single quantities

In [87]:
def get_alpha_beta(elevation_angle, GEO=True):
    """
    Compute the angle between the (satellite-earth center) line and the (satellite-earth surface) line.
    The latter has an elevation_angle given with respect to the earth tangent.
    All angles abre computed in degrees
    """
    
    radius = GEO_RADIUS if GEO else LEO_RADIUS
    
    alpha = to_deg(math.asin(EARTH_RADIUS * sin_deg(90. + elevation_angle) / radius))
    beta = 180. - alpha - 90. - elevation_angle
    
    return alpha, beta
    

In [88]:
def get_range(elevation_angle, GEO=True):
    '''
    Computes the range [meters] of a satellite with a given elevation angle.
    It is possible to define a sat_altitude in case the satellite is not geostationary'''
    
    alti = GEO_ALTITUDE if GEO else LEO_ALTITUDE
    
    alpha, beta = get_alpha_beta(elevation_angle, GEO=GEO)
    return math.sqrt( (alti+EARTH_RADIUS)**2 + EARTH_RADIUS**2 - 2*EARTH_RADIUS*(EARTH_RADIUS+alti) * cos_deg(180-(90+elevation_angle)-alpha)) 

In [89]:
def get_covered_area(elevation_angle, GEO=True):
    
    '''
    Computes the approximation of the covered area (km^2)of a satellite with a given elevation angle
    '''
    
    alpha, beta = get_alpha_beta(elevation_angle, GEO=GEO)
    
    print("alpha = %.2f degree (half angle)" % alpha)
    print("beta = %.2f degree (half angle)" % beta)
    
    
    if (alpha < 10.):
        the_range = get_range(elevation_angle, GEO=GEO)
        r = sin_deg(alpha) * the_range
        return math.pi*r*r
    
    print("The angle Theta/Alpha is over 10° so the approximation made by this function will be incorrect.")
    return -1

In [90]:
def get_orbit_period(orbit_radius):
    '''
    Combutes the orbital period of a satellite in hours'''
    result = 24.*math.pow(orbit_radius/GEO_RADIUS, 1.5)
    pretty_print(result)
    return result

def pretty_print(hour):
    h = int(np.floor(hour))
    frac_h = hour-h
    
    minutes = frac_h * 60
    m = int(np.floor(minutes))
    frac_m = minutes - m
    
    seconds = frac_m * 60
    s = int(np.floor(seconds))
    
    h_str = "hours," if h > 1 else "hour,"
    m_str = "minutes," if m > 1 else "minute,"
    s_str = "seconds." if s > 1 else "second."
    
    print("Duration :", h, h_str, m, m_str, s, s_str)

In [91]:
def get_path_loss(wave_len, the_range, dB=True):
    '''By default the result is in dB'''
    
    result = (wave_len/(4.*math.pi*the_range))**2
    
    if not dB:
        return result
    
    return to_dB(result)
    

In [92]:
def get_anten_area(dish_diameter):
    """Computes the area of a dish antenna given a specified diameter.
    Parameter and output are in meters"""
    return math.pi*dish_diameter*dish_diameter/4.

In [93]:
def get_lambda(transmit_freq, c=3.0*pow(10,8)):
    """Computes the wavelength of a wave with given frequency"""
    return c/transmit_freq

In [94]:
def get_gain_directive(anten_area, wave_len, dB=True):
    """Computes the gain of a directive antenna with given area and knowing the wave length
    of the incoming signal.
    By default the output is in dBi"""
    
    result = 4*math.pi*anten_area/(wave_len*wave_len)
    
    if not dB:
        return result
    
    return to_dB(result)

In [95]:
def get_gain_parabolic(diameter, wavelen, epsilon=0.55, dB= True):
    """Computes the gain of a parabolic dish antenna with given diameter
    and knowing the wave length of the incoming signal.
    By default the output is in dBi"""
    
    result = epsilon * (math.pi*diameter/wavelen)**2
    
    if not dB:
        return result
    
    return to_dB(result)
    

In [96]:
def get_beamwidth_parabolic_3dB(diameter, wavelen, epsilon=0.55, deg=True):
    """Computes the beamwidth of a parabolic dish antenna with a given diameter
    and given the signal wavelength.
    By default the output is in degrees"""
    result = wavelen/(diameter*math.sqrt(epsilon)) 
    
    if not deg:
        return result
    
    return to_deg(result)

In [97]:
def get_EIRP(PT,GT, in_dB=True):
    """Computes the effective isotropic radiated power of a transmitter,
    given its power PT and gain GT. 
    If in_dB== True then PT and GT must be both in dB system, and the output is in dBW.
    Else, PT and GT must be in decimal and the output is in W"""
    
    if in_dB:
        return PT + GT
    
    return PT*GT

In [98]:
def get_Aeff(wavelen, isoG=1.):
    """Computes the effective area with the relative gain to an isotropic radiator"""
    return wavelen*wavelen/(4*math.pi)

In [99]:
def get_flux_density(PR, wavelen, isoG=1., in_dB=True):
    """Computes the flux density with the relative gain to an isotropic radiator.
    If in_dB == True then PR must be in dBW and the output is in dBW/m^2.
    Else, PR must be in W and the output is in W/m^2 """
    
    Aeff = get_Aeff(wavelen, isoG=isoG)
    
    if not in_dB:
        return PR/Aeff
    
    return PR - to_dB(Aeff)

In [100]:
def get_received_power(PT, GT, GR, wavelen, the_range, in_dB=True):
    """Computes the received power for an antenna.
    If in_dB==True then PT, GT, GR must be in db and the output is in dBW
    Else, PT, GT, GR MUST NOT be in dB and the output is in W
    """
    
    const = (wavelen/(4*math.pi*the_range))**2
    
    if not in_dB:
        return PT*GT*GR*const # W
    
    return PT + GT + GR + to_dB(const) # dBW

def get_received_power_EIRP(EIRP, GR, wavelen, the_range, in_dB=True):
    """Computes the received power for an antenna.
    If in_dB==True then EIRP, GR must be in db and the output is in dBW
    Else, PT, GT, GR MUST NOT be in dB and the output is in W
    """
    
    const = (wavelen/(4*math.pi*the_range))**2
    
    if not in_dB:
        return EIRP*GR*const # W
    
    return EIRP + GR + to_dB(const) # dBW
    

In [101]:
def get_figure_of_merit(GR, T, in_dB=True):
    """Computes the ratio GR / T for a receiver.
    If in_dB==True then GR and T must be in db and the output is in dB/K
    Else, GR and T MUST NOT be in dB and the output is in K^-1
    """
    
    if not in_dB:
        return GR/T
    
    return GR - T

In [102]:
def get_nois_from_GT(GR, G_over_T):
    """Computes the noise T knowing GR and  GR/T for a receiver.
    NOTE : Be careful with the units :
        - G_over_T must be in dB/K
        - GR must be in dBi
        - the output is in dBK
    """
    
    return -G_over_T+GR

In [103]:
def get_OFD(EIRP, Lp, uplink_losses, wavelen,  in_dB=True):
    """Computes the Operating Flux Density for a receiver.
    This corresponds to the flux density once the IBO has been applied.
    If IBO was 0 dB then we are operating at maximum power
    If in_dB==True then EIRP, Lp and lossens must be in db and the output is in dBW/m^2
    Else, EIRP, Lp and losses MUST NOT be in dB and the output is in W/m^2
    
    Lp must be a negative dB value
    Losses must be positive dB values
    """
    
    gain = 1./get_Aeff(wavelen)
    
    if in_dB:
        return EIRP + Lp - uplink_losses + to_dB(gain)
    
    return EIRP*Lp*gain/uplink_losses
    

In [104]:
def get_noise_density(nois_temp, in_dB=True):
    """
    Computes the noise density N0 knowing the noise temperatur
    If in_dB is true then nois_temp must be in dbK and the output is in dBW/Hz
    Else, nois_temp must be in K and the output is in W/Hz
    """
    
    k = 1.38*pow(10,-23)
    
    if in_dB:
        return nois_temp + to_dB(k)
    
    return nois_temp*k

In [105]:
def get_C_over_N0(PR, N0, in_dB=True):
    """
    Computes the ration C/N0 which corresponds to C/N multiplied by the bandwidth
    If in_dB is true then PR must be in dBW and N0 in dBW/Hz, and the output is in dBHz
    Else, PR must be in W and N0 in W/Hz and the output is in Hz"""
    
    if in_dB:
        return PR-N0
    
    return PR/N0

In [106]:
def get_C_over_T(C_over_N0, in_dB=True):
    """Computes the ratio C/T knowing C/N0
    If in_dB is True then C_over_N0 must be in dBHz and the output is in dBW/K
    Else, C_over_N0 must be in Hz and the output is in W/K"""
    
    k = 1.38*pow(10,-23)
    
    if in_dB:
        return C_over_N0 + to_dB(k)
    
    return C_over_N0 * k

In [107]:
def get_margin_left(data_R, C_over_N0, req_Eb_over_N0, losses, op_margin, in_dB=True):
    
    """Computes the margin left with specified values
    If in_dB is True then all parameters must be in dB system and so will be the output
    Else, all parameters MUST NOT be in dB system."""
    
    if in_dB:
        return C_over_N0 - (req_Eb_over_N0+losses+op_margin) - data_R
    
    return C_over_N0 / (req_Eb_over_N0*losses*op_margin*data_R)

In [108]:
def get_C_over_N_alldB(PT, GT, GR,  B, Lp, T= to_dB(500), LA=0., LR=0., LM=0.):
    """Computes the ratio C/N.
    Every parameter must be in dB system.
    
    NOTE : LA, LR and LM must be positive dB values
    NOTE : Lp must be a negative dB value"""
    
    if LA<0 or LR<0 or LM<0:
        raise ValueError("LA, LR and LM must be positive dB values.")
        
    if Lp > 0:
        raise ValueError("The path loss Lp must be a negative dB value")
    
    # Boltzmann constants in dB
    k = to_dB(1.38*math.pow(10,-23))
    
    return PT+GT+GR+Lp - (LA+LR+LM+k+T+B)

In [109]:
def get_T_eff(gains,temp):
    """
    Compute the effective temperature in K
    T_eff = T1 + T2/G1 + T3/(G1G2) 
    Parameter : temp values must be in K
    """
    nb_op = min(len(gains)+1,len(temp))
    
    new_gains = np.concatenate([[1],gains[0:nb_op-1]])
    
    # Get the Toeplitz matrix and multiply the elements column wise
    new_gains = np.prod(toeplitz(np.ones([1,nb_op+1]).tolist(),new_gains.tolist()),axis = 0)
    
    # Divide the temperatures by the product of the gain.
    return np.sum(np.array(temp[0:nb_op])/new_gains)

def get_NF(Teff, T0=290.):
    """Computes the receiver noise figure knowing the effective receiver noise"""
    return to_dB(1 + Teff/T0)

def get_system_Noise(cable_loss,gains,temp,Ta):
    
    """Computes the system noise (slide 22 lesson 2-3).
    IMPORTANT : The parameter must be of precise type
        - All values in temp and Ta must be in K.
        - cable_loss must be in dB
    OUTPUT is in K"""
    T0 = 290 #K
    
    Teff = get_T_eff(gains,temp)
    print("Teff =", Teff, "K")
    
    NF = get_NF(Teff)
    print("Noise Figure =", NF, "dB")
    
    
    gamma = 1/to_decimal(cable_loss)
    print("Ts =",gamma*Ta + (1-gamma)*T0+Teff, "K")
    
    return gamma*Ta + (1-gamma)*T0+Teff

In [117]:
def get_Ne(Nd=9):
    return 0.25*(3*Nd*Nd + 1)

In [119]:
def get_d_phased_array(FOV_rad, wavelen, eps=0.55):
    return wavelen / (FOV_rad*np.sqrt(0.55))

## Let's define more complete functions to analyze the link

In [154]:
def oracle(
           GEO = True,
           trans_diam=None,
           rcv_diam = None,
           freq=None,
           wavelen=None,
           elevation_angle = None,
           the_range = None,
           cov_area=None,
           PT=None,
           Lp = None,
           GT=None,
           GR=None,
           G1m2 = None,
           link_losses =0.,
           OFD = None,
           SFD=None,
           IBO=0.,
           OBO=0.,
           flux=None,
           G_over_T=None,
           nois_temp=None,
           N0=None,
           PR=None,
           band=None,
           C_over_N=None,
           C_over_N0=None,
           C_over_T=None,
           data_R=None,
           op_margin=None,
           req_Eb_over_N0=None,
           EIRP=None,
           beam_3dB_rad=None, # radians
           FOV_rad=None, # radians
           Nd=9, # for phased array
           d_phased_array=None,
           ):
    
    """Global oracle, outputs every thing we can know about the system"""

    # ========================================
    # Conversion of the following parameters into dB
    # The data rate, the transmit power, the bandwidth and the noise temperature MUST NOT be in dB system
    data_R_dB = None
    PT_dB = None
    band_dB = None
    nois_temp_dB = None
    
    if data_R is not None:
        data_R_dB = to_dB(data_R)
    
    if PT is not None:
        PT_dB = to_dB(PT)
        
    if band is not None:
        band_dB = to_dB(band)
        
    if nois_temp is not None:
        nois_temp_dB = to_dB(nois_temp)
    
    # ========================================
    
    
    Ne = get_Ne(Nd=Nd)
    
    alpha = None # degree
    beta = None  # degree
    
    if elevation_angle is not None :
        alpha, beta = get_alpha_beta(elevation_angle, GEO=GEO)
    
    if the_range is None and elevation_angle is not None:
        the_range = get_range(elevation_angle, GEO=GEO)
        print("Range = %d km" % (int(the_range)/1000.))
        
    if FOV_rad is None and alpha is not None and beta is not None:
        FOV_rad = to_rad(2*alpha)
        print("FOV = %3f rad" % FOV_rad)
        print("FOV = %3f deg" % to_deg(FOV_rad))
        
    if cov_area is None and elevation_angle is not None:
        cov_area = get_covered_area(elevation_angle, GEO=GEO)/1000000
        print("Covered area = %d km^2 " % int(cov_area))
        
    if FOV_rad is None and alpha is not None and beta is not None:
        FOV_rad = to_rad(2*alpha)
        print("FOV = %3f rad" % FOV_rad)
        print("FOV = %3f deg" % to_deg(FOV_rad))
        
    print()
    
    if wavelen is None and beam_3dB_rad is not None and trans_diam is not None:
        wavelen = np.sqrt(0.55) * beam_3dB_rad * trans_diam
        print("Wavelength =", wavelen, "m")
        
    if wavelen is None and FOV_rad is not None and d_phased_array is not None:
        wavelen = np.sqrt(0.55) * FOV_rad * d_phased_array
        print("Wavelength =", wavelen, "m")
        
    
    if trans_diam is None and wavelen is not None and beam_3dB_rad is not None:
        trans_diam = wavelen / (beam_3dB_rad * sqrt(0.55))
        print("Transmitter diameter = %.2f m" % trans_diam)
        
    
    if freq is not None and wavelen is None:
        wavelen = get_lambda(freq)
        print("Wavelength =", wavelen, "m")
        
    if Lp is None and wavelen is not None and the_range is not None:
        Lp = get_path_loss(wavelen, the_range)
        print("Path loss Lp = %.2f dB" % Lp)
        
    if d_phased_array is None and wavelen is not None and FOV_rad is not None:
        d_phased_array = get_d_phased_array(FOV_rad, wavelen)
        
    from_phased_array = False
    if trans_diam is None and d_phased_array is not None:
        from_phased_array = True
        trans_diam = Nd * d_phased_array
        
    print()
    
    if beam_3dB_rad is None and wavelen is not None and trans_diam is not None:
        beam_3dB_rad = get_beamwidth_parabolic_3dB(trans_diam, wavelen, deg=False)
        print("Beamwidth 3dB = %.4f radians" % beam_3dB_rad)
        print("Beamwidth 3dB = %.4f degrees" % to_deg(beam_3dB_rad))
        
    if FOV_rad is None and wavelen is not None and d_phased_array is not None:
        FOV_rad = get_beamwidth_parabolic_3dB(d_phased_array, wavelen, deg=False)
        print("FOV = %.4f radians" % FOV_rad)
        print("FOV = %.4f degrees" % to_deg(FOV_rad))
        
    print()

    if G1m2 is None and wavelen is not None:
        G1m2 = to_dB(1./get_Aeff(wavelen))
        print("Gain of 1 m^2 = %.2f dBW/m^2" % G1m2)
    
    if GT is None and trans_diam is not None and wavelen is not None:
        GT = get_gain_parabolic(trans_diam, wavelen)
        print("GT = %.2f dBi" % GT)
    
    if GR is None and rcv_diam is not None and wavelen is not None:
        GR = get_gain_parabolic(rcv_diam, wavelen)
        print("GR = %.2f dBi" % GR)
        
    if nois_temp_dB is None and GR is not None and G_over_T is not None:
        nois_temp_dB = get_nois_from_GT(GR, G_over_T)
        print("Ts = %.2f dBK = %.2f K" % (nois_temp_dB, to_decimal(nois_temp_dB)))
        
    if G_over_T is None and GR is not None and nois_temp_dB is not None:
        G_over_T = get_figure_of_merit(GR, nois_temp_dB)
        print("G/T = %.2f dB/K" % G_over_T)
        
    if PR is None and PT_dB is not None and GT is not None and GR is not None and wavelen is not None and the_range is not None:
        PR = get_received_power(PT_dB, GT, GR, wavelen, the_range) -OBO
        print("PR = %.2f dBW" % PR)
        
    if PR is None and EIRP is not None and GR is not None and wavelen is not None and the_range is not None:
        PR = get_received_power_EIRP(EIRP, GR, wavelen, the_range) - OBO
        print("PR = %.2f dBW" % PR)
    
    print()
        
    if N0 is None and nois_temp_dB is not None:
        N0 = get_noise_density(nois_temp_dB)
        print("N0 = %.2f dBW/Hz" % N0)
        
    if C_over_N0 is None and PR is not None and N0 is not None:
        C_over_N0 = get_C_over_N0(PR,N0)
        print("C/N0 = %.2f dBHz" % C_over_N0)
        
    if C_over_T is None and C_over_N0 is not None:
        C_over_T = get_C_over_T(C_over_N0)
        print("C/T = %.2f dBW/K" % C_over_T)
        
    if C_over_N is None and PT_dB is not None and GT is not None and GR is not None and band_dB is not None and Lp is not None and nois_temp_dB is not None:
        C_over_N = get_C_over_N_alldB(PT_dB, GT, GR, band_dB, Lp, LA=link_losses, T=nois_temp_dB)
        print("C/N = %.2f dB" % C_over_N)
        
    if C_over_N0 is not None and req_Eb_over_N0 is not None and link_losses is not None and op_margin is not None and data_R_dB is not None:
        print("Margin left = %.2f dB" % get_margin_left(data_R_dB, C_over_N0, req_Eb_over_N0, link_losses, op_margin))
        
    print()
        
    if flux is None and PR is not None and wavelen is not None:
        flux = get_flux_density(PR, wavelen)
        print("Flux density = %.2f dBW/m^2" % flux)
        
        
    if EIRP is None and GT is not None and PT_dB is not None:
        EIRP = get_EIRP(PT_dB, GT)
        print("EIRP = %.2f dBW" % EIRP)
        
    if OFD is None and EIRP is not None and Lp is not None and wavelen is not None :
        OFD = get_OFD(EIRP, Lp, link_losses, wavelen)
        print("OFD = %.2f dBW/m^2" % OFD)
        
        if SFD is None and IBO is not None:
            print("SFD = %.2f dBW/m^2" % (OFD+IBO))
        
        if SFD is not None:
            print("IBO = %.2f dB" % (SFD-OFD))
            
            
    print("\nIf considering a Phased Array:")
    print("Nd = %d and Ne = %d" % (Nd, Ne))        
    
    if d_phased_array is not None:
        print("d = %.5f m" %d_phased_array)
        
    if from_phased_array:
        print("trans_diam = %.5f m" %trans_diam)
        
    

#### Uplink cell

In [155]:
oracle(freq=6000*pow(10,6),
       elevation_angle=30.,
       PT=75, #W
       trans_diam = 3.4,#m
       rcv_diam=1.,#m
       link_losses=0., # uplink losses
       G_over_T = 9.1, #dB/K
       SFD=-90, #dBW/m^2
       
       IBO = None,
       EIRP=None, # The EIRP concept includes the concept of transmitting antenna and transmitting power
       nois_temp=None,
       req_Eb_over_N0 = None,
       data_R=None,
       band=None)

Range = 38621 km
FOV = 0.262694 rad
FOV = 15.051246 deg
alpha = 7.53 degree (half angle)
beta = 52.47 degree (half angle)
Covered area = 80381113 km^2 

Wavelength = 0.05 m
Path loss Lp = -199.74 dB

Beamwidth 3dB = 0.0198 radians
Beamwidth 3dB = 1.1361 degrees

Gain of 1 m^2 = 37.01 dBW/m^2
GT = 44.00 dBi
GR = 33.37 dBi
Ts = 24.27 dBK = 267.13 K
PR = -103.63 dBW

N0 = -204.33 dBW/Hz
C/N0 = 100.71 dBHz
C/T = -127.89 dBW/K

Flux density = -66.61 dBW/m^2
EIRP = 62.75 dBW
OFD = -99.98 dBW/m^2
IBO = 9.98 dB

If considering a Phased Array:
Nd = 9 and Ne = 61
d = 0.25665 m


#### Downlink cell

In [156]:
oracle(freq=4000*pow(10,6),
       elevation_angle=30.,
       PT=None,
       trans_diam = 1.,
       rcv_diam=0.42,
       nois_temp=385,
       link_losses=6., # miscellaneous (implementation...)
       op_margin = 6., # operating margin (rain, atmosphere)
       OBO = 2.5, # Deduced from IBO of uplink and slide 28 of lecture 2-3
       
       req_Eb_over_N0 = 8.,
       data_R=128*6*pow(10,3),
       
       band=1024*pow(10,3),
       EIRP=53.16,
       SFD = 0.,
       OFD = 0.)
       

Range = 38621 km
FOV = 0.262694 rad
FOV = 15.051246 deg
alpha = 7.53 degree (half angle)
beta = 52.47 degree (half angle)
Covered area = 80381113 km^2 

Wavelength = 0.075 m
Path loss Lp = -196.22 dB

Beamwidth 3dB = 0.1011 radians
Beamwidth 3dB = 5.7943 degrees

Gain of 1 m^2 = 33.49 dBW/m^2
GT = 29.85 dBi
GR = 22.31 dBi
G/T = -3.54 dB/K
PR = -123.25 dBW

N0 = -202.75 dBW/Hz
C/N0 = 79.50 dBHz
C/T = -149.10 dBW/K
Margin left = 0.64 dB

Flux density = -89.76 dBW/m^2

If considering a Phased Array:
Nd = 9 and Ne = 61
d = 0.38497 m


In [157]:
Ts = get_system_Noise(1,[10,30],[300,500],100)

Teff = 350.0 K
Noise Figure = 3.4378197608493104 dB
Ts = 489.07763540238653 K


#### Exercise week 10

In [158]:
oracle(elevation_angle=30,
       PT=10,
       EIRP=50,
       freq=30*pow(10,9),      
       GEO=True)

Range = 38621 km
FOV = 0.262694 rad
FOV = 15.051246 deg
alpha = 7.53 degree (half angle)
beta = 52.47 degree (half angle)
Covered area = 80381113 km^2 

Wavelength = 0.01 m
Path loss Lp = -213.72 dB

Beamwidth 3dB = 0.0292 radians
Beamwidth 3dB = 1.6724 degrees

Gain of 1 m^2 = 50.99 dBW/m^2
GT = 40.64 dBi


OFD = -112.73 dBW/m^2
SFD = -112.73 dBW/m^2

If considering a Phased Array:
Nd = 9 and Ne = 61
d = 0.05133 m
trans_diam = 0.46197 m


In [159]:
0.01/(np.sqrt(0.55)*to_rad(15.05))

0.05133396234876324

In [161]:
oracle(d_phased_array=0.0515, FOV_rad=0.262694)


Wavelength = 0.010033182853650905 m

Beamwidth 3dB = 0.0292 radians
Beamwidth 3dB = 1.6724 degrees

Gain of 1 m^2 = 50.96 dBW/m^2
GT = 40.64 dBi



If considering a Phased Array:
Nd = 9 and Ne = 61
d = 0.05150 m
trans_diam = 0.46350 m
